In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import adaptive
import holoviews as hv
adaptive.notebook_extension()

In [ ]:
import cmath
import math
import operator
from types import SimpleNamespace
import functools

import kwant
import numpy as np
cot = lambda x: 1 / np.tan(x)

def nu_arg(p):
    num = p.E_z * np.exp(1j * p.theta) - 1j * p.alpha * p.k_x
    denom = p.E_z**2 + p.alpha * p.k_x * (p.alpha * p.k_x - 2 * p.E_z * np.sin(p.theta))
    return num / np.sqrt(denom)

def omega(p, j, sign):
    op = {-1: operator.sub,
          +1: operator.add}[sign]
    return 0.25 * op(gamma(p, j), delta(p, j))

def gamma(p, j):
    k = k_j(p, j)
    x = 1j * k * np.tan(p.W * k / 2)
    return (q(p) + x) / (q(p) - x)

def delta(p, j):
    k = k_j(p, j)
    x = 1j * k * cot(p.W * k / 2)
    return (q(p) - x) / (q(p) + x)

def beta(p, sign):
    w_1 = omega(p, 1, sign)
    w_2 = omega(p, 2, sign)
    nu = nu_arg(p)
    beta = np.array([
        [nu * (w_1 - w_2), (w_1 + w_2)],
        [-(w_1 + w_2), nu.conj() * (w_2 - w_1)]
    ])
    return beta

def S(p):
    if isinstance(p, dict):
        p = SimpleNamespace(**p)
    p.E_z = 0.5 * p.mu_B * p.g * p.B
    r_ll = beta(p, +1)
    t_rl = np.exp(-1j * q(p) * p.W) * beta(p, -1)
    t_lr = np.exp(-1j * q(p) * p.W) * beta(p, -1)
    t_rr = np.exp(-2j * q(p) * p.W) * beta(p, +1)

    S = np.block([[r_ll, t_rl],
                  [t_lr, t_rr]])
    return S

def q(p):
    return np.sqrt((2 * p.m_eff / p.hbar**2) * p.mu_n - p.k_x**2)

def k_j(p, j):
    op = {1: operator.sub,
          2: operator.add}[j]
    p.E_z = 0.5 * p.mu_B * p.g * p.B
    sqrt_term = np.sqrt(
        p.E_z**2
        - 2 * p.alpha * p.E_z * np.sin(p.theta) * p.k_x + p.alpha**2 * p.k_x**2)
    k_sq = (2 * p.m_eff / p.hbar**2) * op(p.mu_n, sqrt_term) - p.k_x**2
    return np.sqrt(k_sq)

# B

In [ ]:
import scipy.constants
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (
        scipy.constants.eV * 1e-3))

params = dict(g=26,
              mu_n=40,
              alpha=20,
              B=0.4,
              theta=np.arctan(1/4),
              k_x=0.0,
              W=200,
              phi=np.pi,
              **constants)

k_F = lambda p: np.sqrt(p['mu_n'] * (2 * p['m_eff'])) / p['hbar']

In [ ]:
def a_min_original(z_x, z_y, W):
    theta = np.arctan(4 * z_y / z_x)
    crossing_point = W / np.cos(theta)
    if z_y < crossing_point / 2:
        return np.nan
    a = z_x / 2
    b = z_y * 2
    c = np.sqrt(W**2 + (b/a)**2 * W**2)
    D = np.sqrt(a**2 + (b + c)**2)
    xprime = np.sqrt(D**2 - W**2)
    return W / xprime

def a_min(z_x, z_y, W):
    theta = np.arctan(4 * z_y / z_x)
    crossing_point = W / np.cos(theta)
    if z_y < crossing_point / 2:
        return np.nan
    a = z_x / 2
    b = z_y * 2
    c = np.sqrt(W**2 + (b/a)**2 * W**2)
    D = np.sqrt(a**2 + (b - c)**2)
    xprime = np.sqrt(D**2 - W**2)
    return W / xprime

In [ ]:
def tan_a(params):
    hbar = params['hbar']
    m_eff = params['m_eff']
    alpha = params['alpha']
    Ez = params['g']*params['mu_B']*params['B']
    theta = params['theta']
    kx = params['k_x']
    
    p = SimpleNamespace(**params)
    ky1 = k_j(p, 1)
    ky2 = k_j(p, 2)
    
    ratio_1 = (hbar**2 * ky1 / 
     ((alpha*m_eff*(alpha*kx - Ez*np.sin(theta)))/
      np.sqrt(Ez**2 + alpha*kx*(alpha*kx - 2*Ez*np.sin(theta)))
      + hbar**2*kx)
    )
    ratio_2 = (hbar**2 * ky2 / 
     (-(alpha*m_eff*(alpha*kx - Ez*np.sin(theta)))/
      np.sqrt(Ez**2 + alpha*kx*(alpha*kx - 2*Ez*np.sin(theta)))
      + hbar**2*kx)
    )
    
    return ratio_1, ratio_2
    
def a_max(kx):
    return tan_a(dict(params, k_x=kx))

def a_max_2(kx):
    return np.min(np.abs(tan_a(dict(params, k_x=kx))))

def a_max_fzero(kx):
    a_m = a_max(kx)
    a_p, a_n = (-10.0, 10.0)
    for a in a_m:
        if a < 0:
            a_n = a
        if a >= 0:
            a_p = a
    return min(np.abs((max(a_n, -10.0), min(a_p, 10.0))))

#     return min(np.min(a_max(kx)), 1)

In [ ]:
a_s = [(k, np.min(tan_a(dict(params, k_x=k)))) for k in np.linspace(0.01, .32, 10000)]

In [ ]:
a_min(1000,200,100)

In [ ]:
tan_a(dict(params, k_x=.23))

In [ ]:
def get_cutoff(z_x, z_y, W, params, **_):
    a_m = a_min(z_x, z_y, W)
    
    k_F = np.sqrt(params['mu_n'] * (2 * params['m_eff'])) / params['hbar']
    
    def f(k):
        return abs(np.min(tan_a(dict(params, k_x=k))) - a_m)
    
#     return scipy.optimize.fixed_point(f, k_F/2, xtol=1e-6, maxiter=50)[0]
    k_c, e = scipy.optimize.fmin(f, k_F/2, ftol=1e-7, xtol=1e-6, full_output=1, disp=0)[:2]
    if e < 1e-3:
        return k_c
    else:
        return np.inf

In [ ]:
get_cutoff(1000, 110, 200, params)

# Full kwant system

In [ ]:
import sns_system, spectrum

In [ ]:
params

In [ ]:
params = dict(params, W=200, theta = np.arctan(1/20), mu_n=40)

In [ ]:
a = 1
syst, _, _ = sns_system.make_system(L_m=params['W'],
                              L_x=a,
                              L_sc_up=1600,
                              L_sc_down=1600,
                              z_x=a,
                              z_y=0,
                              a=a,
                              shape=None,
                              transverse_soi=True,
                              mu_from_bottom_of_spin_orbit_bands=False,
                              k_x_in_sc=True,
                              wraparound=True,
                              current=False,
                              ns_junction=False)

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

pars= dict(g_factor_middle = params['g'],
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = params['mu_n'],
                 alpha_middle = params['alpha'],
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 1,
                 Delta_right = 1,
                 B_x = np.cos(params['theta'])*params['B'], B_y = np.sin(params['theta'])*params['B'], B_z = 0,
                 phase = params['phi'],
                 T = 0.0,
                 V = 0,
                 **constants)

In [ ]:
from functools import lru_cache, partial

In [ ]:
def bands(kx, theta):
    return np.min(np.abs(spectrum.calc_spectrum(syst, dict(pars, theta=theta, B_x = np.cos(theta)*params['B'], B_y = np.sin(theta)*params['B'], k_x=kx*a), k=2)[0]))

def learner_bandstructures_for_ratios(ratios):
    cdims = []
    learners = []
    
    k_F = np.sqrt(params['mu_n'] * (2 * params['m_eff'])) / params['hbar']
    
    for ratio in ratios:
        
        theta = np.arctan(4/ratio)
        

        l = adaptive.Learner1D(partial(bands, theta=theta), [0,1.1*k_F], loss_per_interval=adaptive.learner.learner1D.triangle_loss)
        
        learners.append(l)
        cdims.append({'ratio':ratio})
    
    return cdims, adaptive.BalancingLearner(learners)

def process_bandstructures(learner, ratios):
    gaps = {}
    
    for l, ratio in zip(learner.learners, ratios):
        data = l.data
        min_data = np.min(data.values())
        ks, energies = (np.array(list(data.keys())), np.array(list(data.values())))
        idx_sort = np.argsort(ks)
        ks = ks[idx_sort]
        energies = energies[idx_sort]
        
        z_y = np.linspace(100, 400, 100)
        W = params['W']
        k_max = [get_cutoff(ratio*zy, zy, W, params) for zy in z_y]

        temp_energies = np.interp(k_max, ks, np.minimum.accumulate(energies,0))
        gaps[ratio] = scipy.interpolate.interp1d(z_y, temp_energies)
        
    return gaps

def calc_gap_at_ratio_zy(ratio_zy):
    ratio, zy = ratio_zy
    zx = ratio*zy
    
    theta = np.arctan(4/ratio)
    p = dict(pars, theta=theta, B_x=np.cos(theta)*params['B'], B_y=np.sin(theta)*params['B'])
    
    gap_at_k = calc_function_gap_at_k(**p)
    
    return gap_at_k(get_cutoff(ratio*zy, zy, p['W']))

In [ ]:
ratios = np.arange(4, 20, 2)

In [ ]:
cdims, bl = learner_bandstructures_for_ratios(ratios)

In [ ]:
runner = adaptive.Runner(bl)
runner.live_info()

In [ ]:
bl.plot(cdims)

In [ ]:
get_cutoff(6*200, 200, 200, params)

In [ ]:
gaps = process_bandstructures(bl, ratios)

In [ ]:
zs = np.linspace(100, 400, 200)
plt.plot(zs, [gaps[16](zy) for zy in zs])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow((np.vstack([gaps[ratio](z_y) for ratio in ratios])), origin='upper',aspect='auto', extent=(100, 400, 18, 4))

In [ ]:
ratios[::4]

In [ ]:
np.arctan(4/16)/np.pi*180

In [ ]:
z_y = np.linspace(100, 400, 800)

fig, ax = plt.subplots(figsize=(10,10))

ax.plot(z_y, np.array([gaps[ratio](z_y) for ratio in ratios[::]]).T)
# leg = [f'{np.arctan(4/ratio) / np.pi *180:1.0f}°' for ratio in ratios[::]]
# ax.plot([ 100.,  130.,  160.,  190.,  220.,  250.,  280.,  310.,  340.,
#         370.,  400.], 2.8*np.array([0.096164053717058523,
#  0.13874260960712217,
#  0.13585347954578353,
#  0.099580499138667372,
#  0.093469166201382167,
#  0.08052417206107311,
#  0.066261262080846456,
#  0.041305648734786947,
#  0.014092691422112689,
#  0.0043931150876903172,
#  0.011926428763709411]), 'g--')
# ax.plot([ 100.,  130.,  160.,  190.,  220.,  250.,  280.,  310.,  340.,
#         370.,  400.], 2.4*np.array([0.092425946669338677,
#  0.11649105929128199,
#  0.14799343107355897,
#  0.15993370772185847,
#  0.11455261663865851,
#  0.075447385605810754,
#  0.048421488427602882,
#  0.035321998358595612,
#  0.028365663242647811,
#  0.01884507146188618,
#  0.0015759006767117254]), 'C1--')
# ax.plot([ 100.,  130.,  160.,  190.,  220.,  250.,  280.,  310.,  340.,
#         370.,  400.], 1.5*np.array([0.050578059634986114,
#  0.098272143896860203,
#  0.14649981587370489,
#  0.19207651060250597,
#  0.22900959052281952,
#  0.24911963694820546,
#  0.23027806538737206,
#  0.16673564096206356,
#  0.098834430414218369,
#  0.040053066043668491,
#  0.00044228974392221024]),'C0--')

# leg.append(f'{np.arctan(4/8) / np.pi *180:1.0f}°,   simulated zigzag')
# leg.append(f'{np.arctan(4/6) / np.pi *180:1.0f}°,   simulated zigzag')
# leg.append(f'{np.arctan(4/4) / np.pi *180:1.0f}°,   simulated zigzag')
plt.legend(leg);
plt.xlabel('z_y (nm)');
plt.ylabel('gap (E/Δ)');
plt.lo
# plt.savefig('qc_without_conflicted_area_w_full_zz.png', dpi=400)

In [ ]:
theta = np.arctan(4/16)
k_F = np.sqrt(params['mu_n'] * (2 * params['m_eff'])) / params['hbar']
def bands2(kx):
    return np.sort(spectrum.calc_spectrum(syst, dict(pars, k_x=kx*a, B_x=np.cos(theta), B_y=np.sin(theta)), k=2)[0])
learner_real_kwant = adaptive.Learner1D(bands2, [0,1.1*k_F], loss_per_interval=adaptive.learner.learner1D.triangle_loss)

In [ ]:
runner = adaptive.Runner(learner_real_kwant)
runner.live_info()

In [ ]:
a_max(.14)

In [ ]:
params

In [ ]:
def plot(zy):
    k_max = get_cutoff(16*zy, zy, 200, dict(params, alpha=20,theta=np.arctan(4/16)))
    return learner_real_kwant.plot() * hv.VLine(k_max)
    
hv.DynamicMap(plot, kdims=['zy']).redim.range(
    zy=(100, 400))


In [ ]:
data = learner_real_kwant.data
min_data = np.min(data.values())
ks, energies = (np.array(list(data.keys())), np.array(list(data.values())))
idx_sort = np.argsort(ks)
ks = ks[idx_sort]
energies = energies[idx_sort]

gap_at_k = scipy.interpolate.interp1d(ks, np.minimum.accumulate(energies,0))

In [ ]:
# %%opts Curve [logy=True]
hv.Curve((ks, gap_at_k(ks)))

In [ ]:
get_cutoff(ratio*100, 100, W, params)

In [ ]:
ratio = 16
z_y = np.linspace(0, 400)
W = 200
k_max = [get_cutoff(ratio*zy, zy, W, params) for zy in z_y]

In [ ]:
hv.Curve((z_y, gap_at_k(k_max)))

# Plot

In [ ]:
from functools import lru_cache
@lru_cache()
def gap_function(ratio, params=params):
    """Returns a function for the gap(k)."""
    syst = make_syst(W=params['W'])
    params['theta'] = np.arctan(4 / ratio)
    k_max = k_F(params)
    ks = np.linspace(0, k_max, 60)
    energies = [energies_over_delta(syst, params) for params['k_x'] in ks]
    E_abs_mins = [np.min(np.abs(energy)) for energy in energies]
    E_gaps = np.minimum.accumulate(E_abs_mins)
    return scipy.interpolate.interp1d(ks, E_gaps)

In [ ]:
ratios = [4, 8, 16]
gap_funcs = {ratio: gap_function(ratio) for ratio in ratios}

In [ ]:
z_x = 1600
z_y = 100
L = np.sqrt((z_x/2)**2 + (2*z_y)**2)
W = 100

from scipy.misc import derivative
def dk_j(k, j, params):
    p = SimpleNamespace(**params)
    def func(k_x):
        p.k_x = k_x
        return k_j(p, j)
    return derivative(func, k, dx=1e-6, order=7)

def dk_12(k, params):
    return abs(dk_j(k, 1, params)), abs(dk_j(k, 2, params))

def dk_12_analytic(k_x, params):
    from cmath import sqrt, sin, cos
    m_eff, hbar, theta, alpha, mu = [params[k] for k in
        ('m_eff', 'hbar', 'theta', 'alpha', 'mu_n')]
    E_z = 0.5 * params['mu_B'] * params['g'] * params['B']
    c = 2 * m_eff / hbar**2
    sub_sqrt = sqrt(E_z**2 - 2 * alpha * E_z * sin(theta) * k_x + alpha**2 * k_x**2)
    k_1 = sqrt(c * (mu - sub_sqrt) - k_x**2)
    k_2 = sqrt(c * (mu + sub_sqrt) - k_x**2)

    num_num = c * (2 * alpha**2 * k_x - 2 * E_z * alpha * sin(theta))
    num_den = 2 * sqrt(alpha**2 * k_x**2 - 2 * E_z * alpha * k_x * sin(theta) + E_z**2)

    num_1 = - num_num / num_den - 2 * k_x
    num_2 = num_num / num_den - 2 * k_x

    den_1 = 2 * k_1
    den_2 = 2 * k_2

    dk1 = num_1 / den_1
    dk2 = num_2 / den_2
    dk1 = dk1.real if abs(dk1.imag) < 1e-15 else np.nan
    dk2 = dk2.real if abs(dk2.imag) < 1e-15 else np.nan
    return dk1, dk2

def check(dk_12):
    dk1, dk2 = dk_12
    return abs(dk1) < W / L and abs(dk2) < W / L

k_max = k_F(params)
ks = np.linspace(-k_max, k_max, 501)
ks_allowed = np.array([check(dk_12_analytic(k, params=params)) for k in ks])

In [ ]:
ks_allowed

In [ ]:
import matplotlib.pyplot as plt

def cutoff_factor(z_x, z_y, W):
    theta = np.arctan(4 * z_y / z_x)
    crossing_point = W / np.cos(theta)
    if z_y < crossing_point / 2:
        return np.nan
    D = np.sqrt(z_x**2 + (2 * z_y + W * np.sqrt(1 + (4 * z_y/z_x)**2))**2)
    return np.sqrt(1 - (W / D)**2)

def k_cutoff(z_y, ratio, params=params):
    z_x = z_y * ratio
    return k_F(params) * cutoff_factor(z_x, z_y, params['W'])

fig, ax = plt.subplots()
z_ys = np.arange(10, 405, 5)
for ratio in ratios:
    gap_func = gap_funcs[ratio]
    gaps = np.array([gap_func(k_cutoff(z_y, ratio)) for z_y in z_ys])
    ax.plot(z_ys, gaps, label=f'ratio {ratio}')
    ax.legend()

ax.set_ylim(0, 0.2)
plt.show()

In [ ]:
f = gap_funcs[8]
params['theta'] = np.arctan(4 / ratio)
cut = k_F(params)
ks = np.linspace(0, cut)
gaps = np.array([f(k) for k in ks])

dm = hv.DynamicMap(lambda z_y, ratio: hv.VLine(
    k_cutoff(z_y, ratio)), kdims=['z_y', 'ratio']).redim.range(z_y=(10, 400), ratio=(4, 16))

hv.Curve((ks, gaps)) * dm

### Paths

In [ ]:
%%opts Path [show_legend=True]
def plot(B, alpha, mu_n, theta, phi, params=params):
    params['B'] = B
    params['alpha'] = alpha
    params['mu_n'] = mu_n
    params['theta'] = theta
    params['phi'] = phi
    syst = make_syst(W=params['W'])
    k_max = 0.92*k_F(params)
    ks = np.linspace(-k_max, k_max, 50)
    Es_ana = [energies_over_delta(None, params) for params['k_x'] in ks]
    Es_num = [energies_over_delta(syst, params) for params['k_x'] in ks]
    return (hv.Path((ks, Es_ana), label='analytics')[:, -1.1:1.1] 
            * hv.Path((ks, Es_num), label='numerics')[:, -1.1:1.1])

hv.DynamicMap(plot, kdims=['B', 'alpha', 'mu_n', 'theta', 'phi']).redim.range(
    B=(0.01, 1), alpha=(1, 100), mu_n=(10, 50), theta=(0, np.pi), phi=(0, np.pi))